In [1]:
import file_encryptor.convergence
from Crypto.Hash import SHA256
from pyce._crypto import encrypt_path, decryptf
from crypto import ConvergentEncryption
from hashlib import sha256

In [2]:
#from pycryp

In [3]:
#key = convergence.encrypt_inline_file("path/file", None)

In [4]:
text = 'There is some information in the file.'

In [5]:
from Crypto.Cipher import AES
from hashlib import md5
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
from hashlib import sha256

### Generating Convergent Key
Creating Symmetric Key

In [6]:
def ConvKeyGen(hash_value):
    # Using Crypto library
    #key = SHA256.new()
    
    # Encoding the text
    line = hash_value
    #line = line.encode('utf-8')
    print(line)
    
    # Using hashlib library
    #key = sha256(line.encode('utf-8')).digest()
    
    # Making a key
    #key.update(line)
    #key.digest()
    key = md5(line.encode('utf-8')).digest()
    return key

### Encryption using Symmetric Key
We are using AES-CBC. Returns cipher_text, cipher_suite, and tag. 

In [7]:
from cryptography.fernet import Fernet
def ConvEncryption(key, text_file):
    # Generating random blocks
    iv = get_random_bytes(AES.block_size)
    print("This is iv", iv)
    # Creating cipher_suite for random blocks
    cipher_suite = AES.new(key, AES.MODE_CBC, iv)
    #cipher_suite = AES.new(key, AES.MODE_CBC)
    
    # Encrypting text file.
    cipher_text = b64encode(iv + cipher_suite.encrypt(pad(text_file.encode('utf-8'), 
            AES.block_size)))
    #cipher_text = b64encode(cipher_suite.encrypt(pad(text_file.encode('utf-8'), 
     #       AES.block_size)))
    
    
    # Creating a tag
    #cipher_text = cipher_suite.encrypt(text_file.encode('utf-8'))
    tag = md5(cipher_text).digest()
    return cipher_text, cipher_suite, tag
    

### Decryption using Symmetric Key
Using cipher_suit from before

In [8]:
def ConvDecryption(key, cipher_text):
    # Creating a tag
    tag = md5(cipher_text).digest()
    
    # Decrypting the data
    raw = b64decode(cipher_text)
    cipher_suite = AES.new(key, AES.MODE_CBC, raw[:AES.block_size])
    #cipher_suite = AES.new(key, AES.MODE_CBC)
    
    plain_text = unpad(cipher_suite.decrypt(raw[AES.block_size:]), AES.block_size)
    #plain_text = unpad(cipher_suite.decrypt(raw), AES.block_size)
    """
    tag1 = sha256()
    tag1.update(plain_text)
    tag1.digest()
    
    if tag1 == tag:
        return plain_text
    else:
        return "Data is corrupted."
    """
    
    return plain_text

In [9]:
key = ConvKeyGen("Now this is a ciphertext")

Now this is a ciphertext


In [10]:
key

b'l\xcd\xe9\x91@\x82\xb3G\r\xe87a\xdc\xa5\xab\xb9'

In [11]:
cipher_text, cipher_suite, tag = ConvEncryption(key, "Now this is a ciphertext")

This is iv b'(\xd1\x02\xaf*\x1c^\xe9\xcb\x9b\x8d\x89\xedL\xd6\xb8'


In [12]:
cipher_text

b'KNECryocXunLm42J7UzWuENrksNDkG+RU2nmZSH/q3Nx2xl4vTFcHHMWb2cTeQ9X'

In [13]:
ConvDecryption(key, cipher_text)

b'Now this is a ciphertext'

In [14]:
key2 = ConvKeyGen("Something is wrong with this please look into it and let me know. And please improve the secret key")

Something is wrong with this please look into it and let me know. And please improve the secret key


In [15]:
class Counter(object):
    """ 16 Byte binary counter
    
    Example:
        c = Counter()
        c() => \00 * 16
        c() => \00...01
    """
    
    def __init__(self, a=0, b=0, c=0, d=0):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        
    first = True
    def __call__(self):
        if self.first:
            self.first = False
        else:
            if self.d < 0xFFFFFFFF:
                self.d += 1                     # increment byte 0
            elif self.c < 0xFFFFFFFF:
                self.c += 1                     # increment byte 1
                self.d = 0                      # reset byte 0
            elif self.b < 0xFFFFFFFF:
                self.b += 1                     # increment byte 2
                self.c = self.d = 0             # reset bytes 0 and 1
            elif self.a < 0xFFFFFFFF:
                self.a += 1                     # increment byte 3 
                self.b = self.c = self.d = 0    # reset bytes 0, 1, 2
        return struct.pack(">4L", self.a, self.b, self.c, self.d)

## Making a Key using Hash

In [16]:
from crypto import SHA256d
import os, random, struct

### Encryption using Symmetric Key
We are using AES-CBC. Returns cipher_text, cipher_suite, and tag. 

In [17]:
def keygen(data):
    # Need to hash the data to generate a key
    key = sha256(data.encode('utf-8')).digest()
    #h = SHA256d(data.encode('utf-8')).digest()
    
    return key

### Decryption using Symmetric Key
Using cipher_suit from before

In [18]:
def encryption(key, data):
    # AES should be in 16 bit.
    iv = ''.join(chr(random.randint(0, 0xFF)) for i in range(16))

    a = AES.new(key, AES.MODE_CBC, 'This is an IV456'.encode('utf-8'))
    #a = AES.new(key, AES.MODE_CBC, 
    #            iv.encode('utf-8'))
    
    rest = len(data)%16
    if not rest:
        return a.encrypt(data)
    pad = (16-rest)
    #data += chr(pad)*pad
    data += ' ' * (16 - len(data) % 16)

    # Creating a chiphertext
    #cipher_text = a.encrypt(data.encode("utf-8"))[:-pad]
    cipher_text = a.encrypt(data.encode("utf-8"))
    
    # Creating a tag
    tag = sha256(cipher_text).digest()
    
    return cipher_text, a, tag
    
    #return a.encrypt(data.encode("utf-8"))[:-pad]
    

In [19]:
def decryption(key, cipher_text):
    # Creating a tag
    #tag = sha256(cipher_text).digest()
    
    a = AES.new(key, AES.MODE_CBC, 'This is an IV456'.encode('utf-8'))
    plain = a.decrypt(cipher_text)
    
    
    return plain
     

In [20]:
key = keygen('The client sends the locator to the server to store the data. If the server already has the data, it can increment the reference count if desired. If the server does not, the client uploads it. The client need not send the key to the server. (The server can validate the locator without knowing the key simply by checking the hash of the encrypted data.)')
key

b'~s]m\xa8\xd6\xca8\x13\xce\x96\xca\x8e\xa7\x105\xd5\x80\xb4\x1fj\xd5\x0e5<AH\x8a\nU)\x18'

In [21]:
cipher_text, a, tag = encryption(key, 'The client sends the locator to the server to store the data. If the server already has the data, it can increment the reference count if desired. If the server does not, the client uploads it. The client need not send the key to the server. (The server can validate the locator without knowing the key simply by checking the hash of the encrypted data.)')

In [22]:
cipher_text

b'\xbd^\xccI\xe4n\r*\x18\xc38\xde\x85\xe4 \xa7\xaf(\xd0\xb3*\xfa\xee\xc9\xd5Y\x18U\xf5\xac\x95/N\xe3\xa7]\xe6\x85\x82Fs\xd1v\x1b\x19\x11\xdb\x12+2\x13\xde/\xbew\xc8\xff\xfa*D\x1e\xa4\x17\xc9^\xa0WI\xc4\xd5\xf5\xb6\x84\x1c\x1d I\x11\x02\x1e\x16\x993#\xaa\xbd\x9b\xac\xe7J\xb0\xd6\xf0*\xbdi\xb3}\x051Ly\xd2NA\x9c@\xac4\x80\xb7V\xcb\xf6\xac\xd6\xd9\xe2W\xcaEU\x14?\\\xdb9\x05g\xb9\xa6\xd6\x0c\xa8-\x9f?ID\xcdl\xbd\xd4}\xf6Nf5\xf5\xc7\x90W\xa5\xafG\xa9\x84D/\x1eYA.\xa1l\x95\x1f\xf5{q3\xd3\x19\x19}x\xb6\xa0\xef\x0fe+\xaf\xe1\xf0\xdf\xf9O0\x85V\xec\x81\xf6v\xfc\x10f\xfe\x18Z\xdd\xd8\x06\x8f\x94\x85_lr\x8e\xcb_y\xc90x-I\xdbw\x17\xean\xff\t\xe5x\x94&\xec\xcb\x84F\x8fn\xc3\x08\x06Y\xe0s\xdd$I\xbd1\x7fS\xc9\xda\xdat\xcd\xdf\xe1\xc3d\xde\xb3\xd0n?\x9bk\x92"~\xf1\xc3\x7f\x7f/\x82\x17\xd7\x85\x9d\x86\xd3\xce\xa4?\xfb\xb5U\x19\td\x18\xbd\x81\xc5\x8c\xebv%\'\x9a&\xae\x89\n\xe5mZZk)c\x81\xc6\xd9B\x88\xfb\xe3\xc2\x99\x90\xef\xd1a\x8e\r\xd8\xae\n,I#&(\xcb\xe6\x13H\x02+\xe8\xc19\xe7\xf0c\xba\xf0\xa8\xa5\xb0\

In [23]:
decryption(key, cipher_text)

b'The client sends the locator to the server to store the data. If the server already has the data, it can increment the reference count if desired. If the server does not, the client uploads it. The client need not send the key to the server. (The server can validate the locator without knowing the key simply by checking the hash of the encrypted data.)              '

## Creating Hash CE2

In [24]:
class HashCE2:
    
    def __init__(self):
        self.M = M
    
    # Making a new hash
    def newhash(self, M):
        hash = SHA256.new(self.M)
        return hash
    
    # Generating Key
    def generatingKey(self, M):
        key = hash(self.M)
        return key
    
    # Encryption, generating Ciphertext
    def encryption(self, key, M):
        C = convergent(key, self.M)
        # Encryption, Tag generatio
        t = hash(key)
        Cipher = ' '.join(C, t)
        return Cipher
    
    # Decryption, getting message
    def decryption(self, key, Cipher):
        M1 = convergentDecrypt(Cipher, key)
        # Generating the tag
        Tag1 = hash(hash(M1))
        # Checking both tags
        if t != Tag1:
            print("Messsage is Corrupted")
            return None
            
        else:
            self.M = M1
            return self.M

        

### Key Generation
Key generation for HCE2 is same as that for Convergent Encryption

In [25]:
def keygen(data):
    # Need to hash the data to generate a key
    key = sha256(data.encode('utf-8')).digest()
    #h = SHA256d(data.encode('utf-8')).digest()
    
    return key

### Encryption 

In [26]:
def encryptionHce(key, text):
    cipher, a, tag = encryption(key, text)
    tag = sha256(key).digest()
    s = [cipher, tag]
    
    #Cipher = cipher + tag
    Cipher = b''.join(s)
    return Cipher, tag, cipher

### Decryption 

In [27]:
import re
def decryptionHce(cipher, key, tag, Verification = False):
    x = b''+ tag + b'$'
    #tag1 = str(tag)
    #cipher1 = re.sub(tag,'',cipher)
    cipher1 = cipher[:32]
    tag1 = cipher[32:]
    text = decryption(key, cipher1)
    if Verification:
        if tag==tag1:
            return text
        else:
            return "Data is corrupted"
    
    return text
    

In [28]:
key = keygen("There is data in the house.")

In [29]:
Cipher, tag, original = encryptionHce(key, "There is data in the house")

In [30]:
decryptionHce(Cipher, key, tag, Verification = True)

b'There is data in the house      '

In [31]:
Cipher

b'\xa8\xfbW\x1bSQ\xd9\x8b\x81}\xd7\x01\\w^\xbd\x0fU\x08\xe7\x1c*\xecW?\xe6"\x86i\x92\x9a\x979K$T\xd4\x80\x8a\xc1\xf5v\xe99\xf1%\x90\xca\xa6c\x0b\xc4\x10\xa3\xe2\xfa\xe6\x82\x84\xde\xdc\x8cb\xec'

In [32]:
tag

b'9K$T\xd4\x80\x8a\xc1\xf5v\xe99\xf1%\x90\xca\xa6c\x0b\xc4\x10\xa3\xe2\xfa\xe6\x82\x84\xde\xdc\x8cb\xec'

In [33]:
original

b'\xa8\xfbW\x1bSQ\xd9\x8b\x81}\xd7\x01\\w^\xbd\x0fU\x08\xe7\x1c*\xecW?\xe6"\x86i\x92\x9a\x97'

In [34]:
Cipher[:32]

b'\xa8\xfbW\x1bSQ\xd9\x8b\x81}\xd7\x01\\w^\xbd\x0fU\x08\xe7\x1c*\xecW?\xe6"\x86i\x92\x9a\x97'

In [35]:
len(tag)

32